In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install transformers torch
!pip install rouge
!pip install arabert
# Import necessary libraries
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import numpy as np
import nltk
import torch
import pandas as pd
import re
import regex
from tqdm import tqdm
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import MBartForConditionalGeneration, MBartTokenizer
nltk.download("punkt")
nltk.download('stopwords')

# Mount Google Drive if using Colab
from google.colab import drive
drive.mount('/content/drive')

# Path to your labeled validation dataset
file_path = "/content/drive/MyDrive/AIC/labeled_validation_dataset.jsonl"

# Load and preprocess the dataset
df = pd.read_json(file_path, lines=True)
df.drop('example_id', axis=1, inplace=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **MBert Model**

In [ ]:
from transformers import BertTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor

model_name = "malmarjeh/mbert2mbert-arabic-text-summarization"
preprocessor = ArabertPreprocessor(model_name="")

tokenizer = BertTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


# **text summarize**

In [ ]:
def get_summary(text):
    text = preprocessor.preprocess(text)

    max_input_length = tokenizer.model_max_length - tokenizer.num_special_tokens_to_add(pair=True)
    segments = [text[i:i + max_input_length] for i in range(0, len(text), max_input_length)]

    generated_texts = []
    for segment in segments:
        result = pipeline(
            segment,
            pad_token_id=tokenizer.eos_token_id,
            num_beams=3,
            repetition_penalty=3.0,
            max_length=200,
            length_penalty=1.0,
            no_repeat_ngram_size=3
        )[0]['generated_text']
        generated_texts.append(result)

    result = " ".join(generated_texts)
    return result

In [ ]:
# List to store summaries and references
summaries = []
references = []

for sen in tqdm(df["paragraph"][:3]):
    summary_text = get_summary(sen)
    summaries.append(summary_text)

for reference in tqdm(df["summary"]):
    references.append(reference)

  0%|          | 0/3 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
 33%|███▎      | 1/3 [00:32<01:05, 32.84s/it]Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
 67%|██████▋   | 2/3 [00:59<00:29, 29.06s/it]Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.
100%|██████████| 154/154 [00:00<0

In [ ]:
summaries[1]

'دبلوماسيو الدولتين يطالبون بحملة تونس ضد الإجراء الفرنسي 15 دولة عربية تتعهد بالتزام " الاحترام إزاء الإمبراطورية الشريفة " المغرب يعلن عن موقفه من تثبيت فكرة " تنحية السلطان الشرعي " في المنطقة سلطة فرانكو'

In [ ]:
references[1]

'دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قام الأعيان بتوقيعها, ومعارضة الدول العربية والآسيوية كانت تعتبر حملة شرسة ضد الإجراء الفرنسي عن طريق الصحافة, وقام علال الفاسي قال في إذاعة القاهرة عن تنحية السلطان وعائلته عن الوطن, والمعاهدة الذي حدثت بين البلدين كانت تنص بالتزام الإحترام للإمبراطورية الشريفة, وعملية تنصيب سلطان جديد على المغرب في أسبانيا أخذت الرفض.'

# **Rouge**

In [ ]:
from rouge import Rouge
def calculate_rouge(reference, summary):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference)
    return scores[0]

In [ ]:
# حساب مقاييس ROUGE
rouge_scores = []

for reference, summary in zip(references, summaries):
    rouge_score = calculate_rouge(reference, summary)
    rouge_scores.append(rouge_score)

In [ ]:
rouge_scores[1]

{'rouge-1': {'r': 0.20754716981132076, 'p': 0.34375, 'f': 0.25882352471695513},
 'rouge-2': {'r': 0.07142857142857142,
  'p': 0.11764705882352941,
  'f': 0.08888888418765457},
 'rouge-l': {'r': 0.16981132075471697, 'p': 0.28125, 'f': 0.21176470118754337}}

# **Test**

In [ ]:
text = """شهدت مدينة طرابلس، مساء أمس الأربعاء، احتجاجات شعبية وأعمال شغب لليوم الثالث على التوالي، وذلك بسبب تردي الوضع المعيشي والاقتصادي.
 واندلعت مواجهات عنيفة وعمليات كر وفر ما بين الجيش اللبناني والمحتجين استمرت لساعات،
 إثر محاولة فتح الطرقات المقطوعة، ما أدى إلى إصابة العشرات من الطرفين."""

test=get_summary(text)

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


احتجاجات في طرابلس على خلفية مواجهات عنيفة بين الجيش اللبناني والمحتجين
